In [26]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import wandb

In [27]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [28]:
h_params = {
    "epochs":10,
    "learning_rate":0.0001,
    "batch_size":256,
    "num_of_filter":32,
    "filter_size":[5,5,5,5,5],
    "actv_func":"relu",
    "filter_multiplier":1,
    "data_augumentation":False,
    "data_normalization":False,
    "dropout":0,
    "conv_layers":5,
    "dense_layer_size":50
}

IMAGE_SIZE = 224
NUM_OF_CLASSES = 10

In [31]:
#Preparing the data
desired_size = (IMAGE_SIZE, IMAGE_SIZE)

train_transform = transforms.Compose([
    transforms.Resize(desired_size),  
    transforms.ToTensor()        
])

test_transform = transforms.Compose([
    transforms.Resize(desired_size),  
    transforms.ToTensor()        
])

def split_dataset_with_class_distribution(dataset, split_ratio):
    class_indices = {}
    for idx, (image, label) in enumerate(dataset):
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(idx) 

    train_indices = []
    val_indices = []
    for class_idx in class_indices.values():
        print(class_idx)
        split_idx = int(len(class_idx) * split_ratio)
        train_indices.extend(class_idx[:split_idx])
        val_indices.extend(class_idx[split_idx:])

    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)
    return train_dataset, val_dataset


train_data_dir = "/kaggle/input/nature/inaturalist_12K/train"
test_data_dir = "/kaggle/input/nature/inaturalist_12K/val"
train_dataset_total = ImageFolder(train_data_dir, transform=train_transform)
train_dataset, validation_dataset = split_dataset_with_class_distribution(train_dataset_total, 0.8)

test_dataset = ImageFolder(test_data_dir, transform=test_transform)
train_len = len(train_dataset)
val_len = len(validation_dataset)
test_len = len(test_dataset)

batch_size =h_params["batch_size"]
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
test_loader =  DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

TypeError: 'int' object is not callable

In [ ]:

class CNN(nn.Module):
    def __init__(self, h_params):
        super(CNN, self).__init__()
        self.h_params = h_params
        
        #create the no of fileters based on the multiplier
        self.filters =  []
        for i in range(5):
            self.filters.append(self.h_params["num_of_filter"]*(2**i))
        print(self.filters)
        
        # Define convolutional layers
        self.conv_layers = nn.ModuleList()
        in_channels = 3
        for i in range(self.h_params["conv_layers"]):
            conv_layer = nn.Conv2d(in_channels, self.filters[i], self.h_params["filter_size"][i])
            self.conv_layers.append(conv_layer)
            in_channels = self.filters[i]
        
        #Define Fully connected layers
        f_map_side = self.neurons_in_dense_layer(self.h_params["filter_size"], IMAGE_SIZE)
        self.fc1 = nn.Linear( self.filters[-1]*f_map_side*f_map_side , self.h_params["dense_layer_size"])
        self.fc2 = nn.Linear(self.h_params["dense_layer_size"], NUM_OF_CLASSES)
        self.activation_func = self.get_activation_function(self.h_params["actv_func"])

    def forward(self, x):
        for conv_layer in self.conv_layers:
            x = self.activation_func(conv_layer(x))
            x = F.max_pool2d(x, 2, 2)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        
    def get_activation_function(self, activation_func_name):
        if activation_func_name == 'relu':
            return F.relu
        elif activation_func_name == 'gelu':
            return F.gelu
        elif activation_func_name == 'silu':
            return F.silu
        elif activation_func_name == 'leaky_relu':
            return F.leaky_relu
    
    def neurons_in_dense_layer(self, filter_sizes, image_size):
        for i in range(5):
            image_size = int((image_size - filter_sizes[i] +1)/2)
        return image_size

In [ ]:
def train(h_params):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNN(h_params)
    model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=h_params["learning_rate"])


    for epoch in range(h_params["epochs"]):
        training_loss = 0.0
        validation_loss = 0.0
        train_correct = 0
        validation_correct = 0
         # Training phase
        model.train()
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            training_loss+=loss.item()
            values, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()

          
        # Validation phase
        model.eval()
        with torch.no_grad():
            for data in val_loader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = loss_fn(outputs, labels)

                values, predicted = torch.max(outputs, 1)
                validation_correct += (predicted == labels).sum().item()
                validation_loss += loss.item()
        
        train_accuracy = train_correct/train_len
        train_loss  = training_loss/len(train_loader)
        validation_accuracy = validation_correct/val_len
        validation_loss = validation_loss/len(test_loader)
        print("epoch: ", epoch, "train accuray:",train_accuracy , "train loss:",train_loss , "val accuracy:", validation_accuracy,"val loss:",validation_loss)
        
        #logging to wandb
        wandb.log({"train_accuracy":train_accuracy, "train_loss":train_loss, "val_accuracy":validation_accuracy, "val_loss":validation_loss, "epoch":epoch})


    print('Finished Training')
    PATH = './model.pth'
    torch.save(model.state_dict(), PATH)


In [ ]:
config = h_params
run = wandb.init(project="DL Assignment 2", name=f"{config['actv_func']}_ep_{config['epochs']}_lr_{config['learning_rate']}_init_fltr_cnt_{config['num_of_filter']}_fltr_sz_{config['filter_size']}_fltr_mult_{config['filter_multiplier']}_data_aug_{config['data_augumentation']}_data_norm_{config['data_normalization']}_dropout_{config['dropout']}_dense_size_{config['dense_layer_size']}", config=config)
train(config)

In [ ]:
 # Set the model to evaluation mode
model.eval()
correct = 0
with torch.no_grad():  # Disable gradient calculation to speed up computations
  for inputs, labels in train_loader:
      inputs, labels = inputs.to(device), labels.to(device)

      # Forward pass
      outputs = model(inputs)

      # Compute the predicted labels
      values, predicted = torch.max(outputs, 1)

      # Update counts of correct and total predictions
      correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = correct / train_len

print("accuracy: ", accuracy)